<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span></li></ul></div>

# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Описание данных

* id — уникальный идентификатор скважины;
* f0, f1, f2 — три признака точек;
* product — объём запасов в скважине (тыс. баррелей).

# Условия задачи

* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## Загрузка и подготовка данных

Имортируем нужные библиотеки.

In [1]:
import numpy as np
import pandas as pd
from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score

Откроем и просмотрим данные.

In [2]:
df1 = pd.read_csv('/datasets/geo_data_0.csv')
df1.info()
df1.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


In [3]:
df2 = pd.read_csv('/datasets/geo_data_1.csv')
df2.info()
df2.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


In [4]:
df3 = pd.read_csv('/datasets/geo_data_2.csv')
df3.info()
df3.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


Проверим id скважин на дубликаты.

In [5]:
print(df1['id'].value_counts().head())
print(df2['id'].value_counts().head())
print(df3['id'].value_counts().head())

TtcGQ    2
QcMuo    2
bsk9y    2
Tdehs    2
74z30    2
Name: id, dtype: int64
wt4Uk    2
LHZR0    2
bfPNe    2
5ltQ6    2
EiJLT    1
Name: id, dtype: int64
KUPhW    2
Vcm5J    2
xCHr8    2
VF7Jo    2
pnG8N    1
Name: id, dtype: int64


In [6]:
print(df1.query('id == "fiKDv"'))
print()
print(df2.query('id == "bfPNe"'))
print()
print(df3.query('id == "VF7Jo"'))

          id        f0        f1        f2     product
16633  fiKDv  0.157341  1.028359  5.585586   95.817889
90815  fiKDv  0.049883  0.841313  6.394613  137.346586

          id        f0        f1        f2     product
2721   bfPNe -9.494442 -5.463692  4.006042  110.992147
82178  bfPNe -6.202799 -4.820045  2.995107   84.038886

          id        f0        f1        f2     product
11449  VF7Jo  2.122656 -0.858275  5.746001  181.716817
49564  VF7Jo -0.883115  0.560537  0.723601  136.233420


Есть повторения в id скважин, но их не много. Кроме того, у повторяющихся id разные признаки. Удалим их, на качество модели это не должно повлиять.

In [7]:
#Напишем функцию для удаления дубликатов в id
def remove(data):
    id = data['id'].value_counts().reset_index()
    unique_id = id.query('id == 1')['index'].to_list()
    data.query('id in @unique_id', inplace=True)

In [8]:
remove(df1)
print(df1.shape)
remove(df2)
print(df2.shape)
remove(df3)
print(df3.shape)

(99980, 5)
(99992, 5)
(99992, 5)


**Вывод:**

* Имортиртировали библиотеки.
* Просмотрели данные.
* Пропусков и неверных типов данных не обнаружено.
* Были дубликаты в id скважин, но их было не много и они были удалены.

## Обучение и проверка модели

Как и требуется в условиях задачи будем обучать модель линейной регрессии

In [9]:
model = LinearRegression()

Напишем функцию, которая разделит данные на обучающую и валиадционную выборки в соотношении 75:25, обучит модель и сделает предсказания на валидационной выборке.

In [10]:
def get_predictions(data):
    features = data.drop(['id', 'product'], axis = 1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=.25, random_state=777)
    model.fit(features_train, target_train)
    return pd.Series(model.predict(features_valid)), target_valid

Сохраним предсказания и правильные ответы.

In [11]:
predicted_1, target_1 = get_predictions(df1)
predicted_2, target_2 = get_predictions(df2)
predicted_3, target_3 = get_predictions(df3)

Посчитаем средний объем предсказанного сырья и RMSE модели.

In [12]:
def mean_product_and_rmse(answers, predictions):
    mse = mean_squared_error(answers, predictions)
    print('Средий запас сырья: ', predictions.mean())
    print('RMSE: ', mse ** 0.5)

In [13]:
print('Регион №1')
print()
mean_product_and_rmse(target_1, predicted_1)

Регион №1

Средий запас сырья:  92.36563599165335
RMSE:  37.795050080808814


In [14]:
print('Регион №2')
print()
mean_product_and_rmse(target_2, predicted_2)

Регион №2

Средий запас сырья:  69.16492157800474
RMSE:  0.8899440381915138


In [15]:
print('Регион №3')
print()
mean_product_and_rmse(target_3, predicted_3)

Регион №3

Средий запас сырья:  95.15168921330066
RMSE:  40.053537366877244


**Вывод:**

* Разбили данные на обучающую и валадционную выборки.
* Обучили модель, сделали и сохранили предсказания и ответы.
* Качественнее всего модель отработала на регионе №2, здесь значение RMSE самое маленькое.
* Самый большой запас сырья в регионах №1 и №3.

## Подготовка к расчёту прибыли

В условиях задачи нам даны константы.

In [16]:
# Бюджет на разработку скважин в одном регионе
BUDGET = 10000000000

# Количество скважин для бурения
OIL_WELLS = 200

# Бюджет на одну скважину
BUDGET_ONE_WELL = BUDGET / OIL_WELLS

# Доход с одной единицы продунка
REVENUE_RUB = 450000

# Максимальная доля убытков
MAX_RISK = 0.025

Посчитаем минимальное среднее количество продукта, необходимое, что бы проект был прибыльным.

In [17]:
BUDGET_ONE_WELL / REVENUE_RUB

111.11111111111111

Исходя из расчета в каждом регионе недостаточно сырья для того, что бы проект хотя бы вышел в ноль. Но мы будем исследовать только 500 скважин, из которых выберем 200 самых прибыльных. 

Напишем функцию для расчета прибыли по 200 скважин каждого региона из которых получится добыть больше всего сырья.

In [18]:
def profit(target, probabilities):
    target = pd.Series(target).reset_index(drop=True)
    probabilities = pd.Series(probabilities).reset_index(drop=True)
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:OIL_WELLS]
    return (((REVENUE_RUB * selected.sum()) - BUDGET) / 1000000000).round(2)

In [19]:
profit(target_1, predicted_1)

3.1

In [20]:
profit(target_2, predicted_2)

2.42

In [21]:
profit(target_3, predicted_3)

2.3

Судя из расчетов, самым прибыльм является регион №1, c него можно получить самую большую выручку с добычи сырья - 3,1 млрд. рублей.

## Расчёт прибыли и рисков 

Посчитаем риски и прибыль каждого региона с помощью бутстрепа.

In [22]:
state = np.random.RandomState(777)

In [23]:
def bootstrap(target, predictions):
    value = []
    for i in range(1000):
        target_subsample = target.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
        probs_subsample = predictions[target_subsample.index]
        value.append(profit(target_subsample, probs_subsample))
        
    value = pd.Series(value)
    lower = value.quantile(MAX_RISK)
    upper = value.quantile(0.975)
    #confidence_interval = st.t.interval(0.95, len(value)-1, value.mean(), value.sem())
    mean = value.mean()
    
    print('Средняя выручка: ', mean)
    print('2.5% квантиль: ', lower)
    print('97.5% квантиль:', upper)
    #print('Доверительный интервал: ', confidence_interval)
    print('Риски убытка: ', (value < 0).mean())
    


In [24]:
bootstrap(target_1, predicted_1)

Средняя выручка:  0.39266
2.5% квантиль:  -0.15
97.5% квантиль: 0.91
Риски убытка:  0.072


In [25]:
bootstrap(target_2, predicted_2)

Средняя выручка:  0.47969000000000006
2.5% квантиль:  0.09
97.5% квантиль: 0.86
Риски убытка:  0.01


In [26]:
bootstrap(target_3, predicted_3)

Средняя выручка:  0.35902000000000006
2.5% квантиль:  -0.21
97.5% квантиль: 0.91
Риски убытка:  0.096


**Вывод**

Из всех регионов по уровню риска подходит только регион №2. К тому же добыча во втором регоне принесет самую большую прибыль.